# Introduction 

Many businesses very often face the next problem: when it begins to grow up, owners start to think about: where to open new outlets or found another place where you satisfy your customers' needs. This issue usually relates to offline companies like barbershops, veterinary clinics, retail stores, etc. There is a lot of parameters that impact on the decision: monthly rent, competitors, crime rates, a wealth of the people around and many others. This little research is focusing on the similarity of the boroughs or How to split all districts into groups. 

The idea is based on the underlying assumption that we should found spots in similar places.  

For this research, I selected the Greater Sydney Area. It's one of the biggest agglomerations in the world,  consists of almost about 650 suburbs that sprawl about 70 km to the west, 40 km to the north, and 60 to the south and generates approximately 24.1% of Australia's GDP. 

# Data

For classifying suburbs, we should get features for each district. In this research, I will use the next:

* latitude and longitude to show them on a map;
* venue data, amount of each type of venues;
* crime data, about of crimes grouped by type(murder, robbery, carjacking;
* distance from the Sydney Central Business District (CBD).

**Data sources:**

* First we should get the list of suburbs by scrapping the wiki page -  https://en.wikipedia.org/wiki/List_of_Sydney_suburbs. 
* Then get the locations using OpenCage API - https://opencagedata.com/api
* Monthly crime reports from the Bureau of Crime Statistics and Research. There are reporst for each suburb in New South Wales, so we should select only ones that we scrapped on the first step - https://www.bocsar.nsw.gov.au/Pages/bocsar_datasets/Datasets-.aspx
* The list of venues using Foursquare API - https://foursquare.com/;
* Crude distance from the central district will be calculated based on the centres of suburbs, got on the second step.



### Base geodata

In [104]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import re
import requests # library to handle requests
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from geopy.distance import distance # module for calculating distances between suburbs
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
import folium # plotting library
import os
import sys
import googlemaps
import pprint
from datetime import datetime

In [5]:
# Load environment variables from the .env files
%reload_ext dotenv
%dotenv

Scrap suburbs names from the wiki-page

In [138]:
PAGE_URL = 'https://en.wikipedia.org/wiki/List_of_Sydney_suburbs'
contents = urllib.request.urlopen(PAGE_URL).read()
soup = BeautifulSoup(contents, 'html.parser')
block_with_groups = soup.find('div', { "class" : "mw-parser-output"})

listOfSuburbs = []

for h2 in block_with_groups.select('h2'):
    p = h2.findNextSibling('p')
    if p:
        listOfSuburbs = listOfSuburbs + list(map(lambda x: x.string, p.select('a')))

listOfSuburbs = np.unique(np.array(listOfSuburbs))
suburbs_df = pd.DataFrame(listOfSuburbs, columns=['suburb'])
suburbs_df.head()

,suburb
0,Abbotsbury
1,Abbotsford
2,Acacia Gardens
3,Agnes Banks
4,Airds


In [139]:
suburbs_df.shape

(689, 1)

Let's find location for each suburb using the opencage API. Free trial users of the OpenCage API are limited to 1 request per second, so for 700 suburbs this request will be lasting near the 12 minutes. Because of it I savet this dataframe to the csv file that I will use next time. This step can be omitted if csv file already exists.

In [83]:
geocoder = OpenCageGeocode(os.getenv("OPENCAGE_API_KEY"))
latitudes = {}
longitudes = {}
suburbs_with_loc_fdata = []
for suburb in suburbs_df['suburb']:
    result = geocoder.geocode('Australia, New South Wales, ' + suburb,  no_annotations='1')
    long = result[0]['geometry']['lng']
    lat  = result[0]['geometry']['lat']    
    suburbs_with_loc_fdata.append([suburb, lat, long])

suburbs_df = pd.DataFrame(suburbs_with_loc_fdata, columns=['suburb', 'lat', 'long'])
suburbs_df.to_csv(r'suburbs_with_coordinates.csv', index = False)
suburbs_df.head()

,suburb,lat,long
0,Abbotsbury,-33.869285,150.866703
1,Abbotsford,-33.850553,151.129759
2,Acacia Gardens,-33.732459,150.912532
3,Agnes Banks,-33.614508,150.711448
4,Airds,-34.090000,150.826111


In [12]:
gmaps = googlemaps.Client(key=os.getenv('GMAPS_API_KEY'))
resuls = gmaps.geocode('New South Wales, Cendral Business District', components={'country': 'AU'})
latitudes = {}
longitudes = {}
suburbs_with_loc_fdata = []
for suburb in suburbs_df['suburb']:
    result = gmaps.geocode('New South Wales, ' + suburb, components={'country': 'AU'})
    lng = result[0]['geometry']['location']['lng']
    lat  = result[0]['geometry']['location']['lat']    
    suburbs_with_loc_fdata.append([suburb, lat, lng])

suburbs_df = pd.DataFrame(suburbs_with_loc_fdata, columns=['suburb', 'lat', 'lng'])
suburbs_df.to_csv(r'suburbs_with_coordinates.csv', index = False)
suburbs_df.head()

,suburb,lat,lng
0,Abbotsbury,-33.87500,150.86200
1,Abbotsford,-33.85215,151.12726
2,Acacia Gardens,-33.73220,150.91700
3,Agnes Banks,-33.61529,150.71616
4,Airds,-34.09160,150.82490


Read suburb with coordinates from the csv file

In [140]:
suburbs_df = pd.read_csv('suburbs_with_coordinates.csv')
suburbs_df.set_index('suburb', inplace=True)
suburbs_df.head()

,lat,lng
suburb,,
Abbotsbury,-33.87500,150.86200
Abbotsford,-33.85215,151.12726
Acacia Gardens,-33.73220,150.91700
Agnes Banks,-33.61529,150.71616
Airds,-34.09160,150.82490


and show them on the map.

In [141]:
cbd = suburbs_df.loc['Central Business District']
venues_map = folium.Map(location=[cbd['lat'], cbd['lng']], zoom_start=9)
folium.Marker([cbd['lat'], cbd['lng']], popup='CBD', icon=folium.Icon(color='red')).add_to(venues_map)
df = suburbs_df.drop(['Central Business District']) 
df.head()
for lat, lng, label in zip(df.lat, df.lng, df.index):
     folium.Marker(
        [lat, lng],
        popup=label,
        icon=folium.Icon(color='green')
     ).add_to(venues_map)
    
venues_map

Calculate the distance from the Central Busines District

In [142]:
cbd_coordinate = (cbd['lat'], cbd['lng'])

distances = []

for _, row in suburbs_df.iterrows():
    lat = row['lat']
    lng = row['lng']
    distances.append(distance(cbd_coordinate,(lat,lng)).km)

suburbs_df['dist_from_cbd'] = distances

suburbs_df.sort_values(['dist_from_cbd'],inplace = True)

suburbs_df.tail()

,lat,lng,dist_from_cbd
suburb,,,
The Slopes,-33.532801,150.706912,59.649490
Kurmond,-33.551000,150.690000,59.650771
Kurrajong,-33.550000,150.666700,61.464918
Kurrajong Hills,-33.533333,150.650000,63.804356
Nelson,-32.715000,152.151111,155.437429


In [144]:
suburbs_df.tail(10)

,lat,lng,dist_from_cbd
suburb,,,
Yarramundi,-33.622200,150.670900,56.843291
Grose Wold,-33.598611,150.684722,57.072646
Tennyson,-33.536944,150.737500,57.172222
Wisemans Ferry,-33.381944,150.985000,58.018323
Grose Vale,-33.584000,150.674000,58.782859
The Slopes,-33.532801,150.706912,59.649490
Kurmond,-33.551000,150.690000,59.650771
Kurrajong,-33.550000,150.666700,61.464918
Kurrajong Hills,-33.533333,150.650000,63.804356


and display the top 30 farest from the CBD suburbs.

In [51]:
venues_map = folium.Map(location=[-33.299276, 151.407568], zoom_start=8.4)
folium.Marker([cbd['lat'], cbd['lng']], popup='CBD', icon=folium.Icon(color='red')).add_to(venues_map)
df = suburbs_df.tail(30)
df.head()
for lat, lng, dist, label in zip(df.lat, df.lng, df.dist_from_cbd, df.index):
     folium.Marker(
        [lat, lng],
        popup= "%s \n %f km." % (label, dist),
        icon=folium.Icon(color='green')
     ).add_to(venues_map)
    
venues_map

We can see on this map that we have one outliner - Nelson. Let's drop it.

In [147]:
#suburbs_df.drop('Nelson', inplace=True)
suburbs_df.tail(10)

,lat,lng,dist_from_cbd
suburb,,,
East Kurrajong,-33.515278,150.767778,56.711739
Yarramundi,-33.622200,150.670900,56.843291
Grose Wold,-33.598611,150.684722,57.072646
Tennyson,-33.536944,150.737500,57.172222
Wisemans Ferry,-33.381944,150.985000,58.018323
Grose Vale,-33.584000,150.674000,58.782859
The Slopes,-33.532801,150.706912,59.649490
Kurmond,-33.551000,150.690000,59.650771
Kurrajong,-33.550000,150.666700,61.464918


### Crime data

Create dataframe from previously unpacked csv file

In [65]:
crime_df = pd.read_csv('suburb_crime_2019.csv')
crime_df.head()

,Suburb,Offence category,Subcategory,Jan 1995,Feb 1995,Mar 1995,Apr 1995,May 1995,Jun 1995,Jul 1995,...,Mar 2019,Apr 2019,May 2019,Jun 2019,Jul 2019,Aug 2019,Sep 2019,Oct 2019,Nov 2019,Dec 2019
0,Aarons Pass,Homicide,Murder *,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Aarons Pass,Homicide,Attempted murder,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Aarons Pass,Homicide,"Murder accessory, conspiracy",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Aarons Pass,Homicide,Manslaughter *,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Aarons Pass,Assault,Domestic violence related assault,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
crime_df.shape

(276830, 303)

Group them by the offence category and suburb and sum all months

In [66]:
crime_df['SumCrimes'] = crime_df.drop(['Suburb', 'Offence category', 'Subcategory'], axis=1).sum(axis=1)

In [67]:
crime_df_s = crime_df[['Suburb', 'Offence category', 'Subcategory', 'SumCrimes']]
crime_df_s.head()

,Suburb,Offence category,Subcategory,SumCrimes
0,Aarons Pass,Homicide,Murder *,0
1,Aarons Pass,Homicide,Attempted murder,0
2,Aarons Pass,Homicide,"Murder accessory, conspiracy",0
3,Aarons Pass,Homicide,Manslaughter *,0
4,Aarons Pass,Assault,Domestic violence related assault,5


Rename the columns to be able to join with suburb_df

In [88]:
crime_df_s.rename(columns={'Suburb':'suburb', 'Offence category':'offence_category', 'SumCrimes':'sum_crimes'}, inplace=True)
crime_df_s.head()

/home/evan/env/lib/python3.8/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,suburb,offence_category,Subcategory,sum_crimes
0,Aarons Pass,Homicide,Murder *,0
1,Aarons Pass,Homicide,Attempted murder,0
2,Aarons Pass,Homicide,"Murder accessory, conspiracy",0
3,Aarons Pass,Homicide,Manslaughter *,0
4,Aarons Pass,Assault,Domestic violence related assault,5


In [89]:
crime_df_with_category = crime_df_s.groupby(['suburb', 'offence_category']).sum()
crime_df_sum = crime_df_with_category.groupby(['suburb']).sum()
crime_df_sum.head()

,sum_crimes
suburb,
Aarons Pass,80
Abbotsbury,3715
Abbotsford,6009
Abercrombie,597
Abercrombie River,82


And finally we should filter them by the suburb

In [148]:
suburbs_crimes_df = crime_df_sum.merge(suburbs_df, on='suburb')
suburbs_crimes_df.head()

,sum_crimes,lat,lng,dist_from_cbd
suburb,,,,
Abbotsbury,3715,-33.87500,150.86200,31.953973
Abbotsford,6009,-33.85215,151.12726,7.693985
Acacia Gardens,1716,-33.73220,150.91700,30.971957
Agnes Banks,947,-33.61529,150.71616,53.617942
Airds,22071,-34.09160,150.82490,42.992898


### Getting venues from the Forsquare 

In [108]:
CLIENT_ID = os.getenv("FOURSQUARE_CLIENT_ID") # your Foursquare ID
CLIENT_SECRET = os.getenv("FOURSQUARE_CLIENT_SECRET") # your Foursquare Secret
VERSION = '20200401'
RADIUS = 800

venues_list=[]

for suburb, lat, lng in zip(suburbs_df.index, suburbs_df.lat, suburbs_df.lng):
    print(suburb)
    
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        RADIUS, 
        100)

    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    print(len(results))
    
    for venue in results:        
        venues_list.append([
            suburb, 
            lat, 
            lng, 
            venue['venue']['name'], 
            venue['venue']['categories'][0]['name']
        ])

venues = pd.DataFrame(venues_list)
venues.columns = ['suburb', 'lat', 'long', 'venue', 'venue_category']
venues.head()

Central Business District
100
Barangaroo
100
Pyrmont
100
Millers Point
100
Haymarket
100
The Rocks
100
Woolloomooloo
100
Darlinghurst
100
Potts Point
100
Ultimo
100
Dawes Point
100
Surry Hills
100
Chippendale
100
Elizabeth Bay
100
Rushcutters Bay
100
Balmain East
31
Glebe
70
Paddington
74
Darlington
100
Redfern
100
Kirribilli
60
Milsons Point
84
Darling Point
12
Forest Lodge
78
Edgecliff
56
McMahons Point
66
Balmain
73
Moore Park
49
Lavender Bay
100
Eveleigh
94
Camperdown
65
Birchgrove
55
Double Bay
38
Waterloo
82
Kurraba Point
16
North Sydney
87
Centennial Park
51
Rozelle
53
Waverton
22
Annandale
46
Woollahra
29
Cremorne Point
10
Lilyfield
27
Newtown
100
Point Piper
12
Zetland
48
Neutral Bay
70
Erskineville
75
Bellevue Hill
7
Wollstonecraft
20
Alexandria
40
Beaconsfield
50
Bondi Junction
56
Kensington
25
Stanmore
29
Greenwich
14
Woolwich
16
Leichhardt
78
Enmore
77
Crows Nest
94
Queens Park
21
Cremorne
72
Cammeray
19
Drummoyne
23
Rose Bay
45
St Leonards
100
Waverley
18
Bondi
36
Northwo

1
Blair Athol
4
Claremont Meadows
3
Bradbury
4
Blairmount
0
Gregory Hills
4
McGraths Hill
4
Windsor Downs
0
Shanes Park
1
Bringelly
0
Berkshire Park
0
Ambarvale
7
Cattai
2
Werrington
1
Englorie Park
11
Werrington County
6
Currans Hill
5
Orchard Hills
1
Caddens
0
Pitt Town
1
Mulgrave
2
South Maroota
0
Oran Park
4
Smeaton Grange
5
St Helens Park
4
Luddenham
0
Wedderburn
1
Bligh Park
4
Cambridge Park
9
Singletons Mill
0
Windsor
10
Glen Alpine
5
Pitt Town Bottoms
0
South Windsor
5
Rosemeadow
6
Llandilo
2
Mount Annan
5
Werrington Downs
3
Kingswood Park
10
Kingswood
10
Narellan Vale
5
Jordan Springs
4
Harrington Park
5
Cambridge Gardens
6
Cornwallis
0
South Penrith
4
Clarendon
4
Wilberforce
6
Gilead
0
Cranebrook
4
Ebenezer
0
Kirkham
1
Penrith
48
Glenmore Park
10
Greendale
0
Elderslie
5
Menangle Park
2
Londonderry
2
Spring Farm
2
Maroota
0
Jamisontown
22
Cobbitty
4
Freemans Reach
3
Mulgoa
7
Camden
7
Laughtondale
0
Regentville
4
Richmond
15
Camden South
2
Hobartville
2
Emu Plains
6
Ellis Lane


,suburb,lat,long,venue,venue_category
0,Central Business District,-33.870846,151.20733,Kinokuniya,Bookstore
1,Central Business District,-33.870846,151.20733,State Theatre,Theater
2,Central Business District,-33.870846,151.20733,The Baxter Inn,Whisky Bar
3,Central Business District,-33.870846,151.20733,Queen Victoria Building (QVB),Shopping Mall
4,Central Business District,-33.870846,151.20733,Single O,Coffee Shop


In [125]:
sum_venues = venues.groupby('suburb')['venue_category'].value_counts().unstack().fillna(0)
sum_venues.head()

venue_category,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Aquarium,Arcade,Arepa Restaurant,...,Warehouse Store,Water Park,Waterfall,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
suburb,,,,,,,,,,,,,,,,,,,,,
Abbotsbury,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abbotsford,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Acacia Gardens,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Airds,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alexandria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
sum_venues.shape

(646, 376)

And finally, let's join crimes and venues datasets

In [136]:
sum_venues.merge(suburbs_crimes_df, on='suburb')

,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Aquarium,Arcade,Arepa Restaurant,...,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,sum_crimes,lat,lng,dist_from_cbd
suburb,,,,,,,,,,,,,,,,,,,,,
Abbotsbury,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3715,-33.875000,150.862000,31.953973
Abbotsford,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,6009,-33.852150,151.127260,7.693985
Acacia Gardens,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1716,-33.732200,150.917000,30.971957
Airds,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,22071,-34.091600,150.824900,42.992898
Alexandria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,31276,-33.908027,151.190258,4.416159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yagoona,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,31035,-33.907311,151.020482,17.751245
Yarrawarrah,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1628,-34.055200,151.032100,26.085454
Yennora,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7090,-33.862700,150.966000,22.348236
